In [1]:
import os
import sys
import pandas as pd
import numpy as np
import regex as re
import mygene
from  TCGA_files import queryFile
from matplotlib import pyplot as plt

In [2]:
mg = mygene.MyGeneInfo()

In [3]:
#mg.getgene("ENSG00000135094", 'name,symbol,refseq.rna,type_of_gene,exac.bp,genomic_pos')

In [4]:
working_dir = "/Users/filippo/Developer/tesi"
os.chdir(working_dir)
dirs = os.listdir("data")

In [5]:
len(dirs)

4497

In [6]:
df = pd.read_csv(("%s/genes.txt"%working_dir), header=[0], index_col=[0])
df.columns=["type_of_gene"]
print(len(df))
df.head()

60483


,type_of_gene
ENSG00000242268,intergenic
ENSG00000270112,unknown
ENSG00000167578,protein-coding
ENSG00000273842,unknown
ENSG00000078237,protein-coding


In [7]:
uniq = [g[:15] for g in df.index.values]

In [8]:
len(np.unique(uniq))

60483

In [ ]:
#q_many = mg.getgenes(df[df['type_of_gene']=='protein-coding'].index.values,'symbol,genomic_pos,genomic_pos_hg19')

In [20]:
q_many = pd.read_csv("genes_l.txt", index_col=[0], header=[0])
#df_lenghts = df[df['type_of_gene']=='protein-coding']
lenghts = q_many['lenght']
df_lenghts = df
df_lenghts.insert(0,'lenght',lenghts)
df_lenghts.to_csv("genes_l.txt",header=True,index=True)

In [26]:
df = pd.read_csv(("%s/mainTable.csv"%working_dir), index_col=[0])

In [29]:
df.head()

,ab30d37c-c510-496f-a704-6687ea39b847.htseq.counts.gz,e89c96d6-a2ab-4bac-bbba-8c52d49a3edf.htseq.counts.gz,d2587070-cb7d-440d-ae49-52f5077248e6.htseq.counts.gz,e733093b-8a60-46d7-9ddd-ec8a82206126.htseq.counts.gz,f02be12f-2e34-4089-be5c-0f81ca6038c8.htseq.counts.gz,6b424e38-1004-4434-be48-ee01dad67982.htseq.counts.gz,6f8c77f2-67de-411d-a03d-5fef3216b71b.htseq.counts.gz,3490b791-16ec-41e8-9015-5f3d33c856c0.htseq.counts.gz,97675f21-ce94-4e7c-bc46-179cc2903bf3.htseq.counts.gz,df692bf8-d523-4528-aa0c-a0549373adee.htseq.counts.gz,...,87968a34-cdba-4335-92d9-0f5b34ed27c9.htseq.counts.gz,32f7779d-9065-4241-ad44-efe90148ac23.htseq.counts.gz,358f2893-02b2-4d36-9c76-b818c28b6027.htseq.counts.gz,11797830-6218-43c6-8c41-515583575668.htseq.counts.gz,8677d42c-aef1-444d-b9a2-0dbcced0e886.htseq.counts.gz,fbef38b2-454d-44e6-b27f-5a790c6f8c17.htseq.counts.gz,c327832d-9cf8-4734-a745-42879ebe6702.htseq_counts.txt.gz,90d19c5c-f80b-454b-b776-99d13f01617e.htseq.counts,6d9c73d6-215a-49f9-bc9f-af019f8db55c.htseq.counts.gz,9c9ee7b5-0227-43c2-be55-909751dbd61a.htseq.counts.gz
ENSG00000167578,475,494,1008,265,325,278,582,613,357,361,...,901,269,458,1395,621,384,403,697,369,392
ENSG00000078237,409,475,1082,660,804,535,1215,1226,751,675,...,485,651,154,812,674,73,395,598,1240,1090
ENSG00000146083,3211,4264,5319,3457,3864,1257,3702,3043,4536,3259,...,2849,2141,1071,2456,4049,1466,1933,2840,2657,5324
ENSG00000158486,53,18,168,12,116,62,97,18,90,836,...,425,198,0,27,25,790,28,4,153,318
ENSG00000198242,11411,20733,15310,32889,11328,27760,15145,25741,18640,27510,...,8388,14479,12937,24780,27905,15907,4641,24878,11621,39908


In [30]:
ngenes = len(df.index)
nfiles = len(df.columns)
print("genes:%d\trealizations:%d"%(ngenes,nfiles))

genes:19369	realizations:3634


## Counts to RPK
RPK=$\frac{counts}{lenght}$

In [31]:
df = pd.concat((df,q_many), ignore_index=False, sort=False, axis=1)
df.drop('type_of_gene',axis=1,inplace=True)
df.head()

,ab30d37c-c510-496f-a704-6687ea39b847.htseq.counts.gz,e89c96d6-a2ab-4bac-bbba-8c52d49a3edf.htseq.counts.gz,d2587070-cb7d-440d-ae49-52f5077248e6.htseq.counts.gz,e733093b-8a60-46d7-9ddd-ec8a82206126.htseq.counts.gz,f02be12f-2e34-4089-be5c-0f81ca6038c8.htseq.counts.gz,6b424e38-1004-4434-be48-ee01dad67982.htseq.counts.gz,6f8c77f2-67de-411d-a03d-5fef3216b71b.htseq.counts.gz,3490b791-16ec-41e8-9015-5f3d33c856c0.htseq.counts.gz,97675f21-ce94-4e7c-bc46-179cc2903bf3.htseq.counts.gz,df692bf8-d523-4528-aa0c-a0549373adee.htseq.counts.gz,...,32f7779d-9065-4241-ad44-efe90148ac23.htseq.counts.gz,358f2893-02b2-4d36-9c76-b818c28b6027.htseq.counts.gz,11797830-6218-43c6-8c41-515583575668.htseq.counts.gz,8677d42c-aef1-444d-b9a2-0dbcced0e886.htseq.counts.gz,fbef38b2-454d-44e6-b27f-5a790c6f8c17.htseq.counts.gz,c327832d-9cf8-4734-a745-42879ebe6702.htseq_counts.txt.gz,90d19c5c-f80b-454b-b776-99d13f01617e.htseq.counts,6d9c73d6-215a-49f9-bc9f-af019f8db55c.htseq.counts.gz,9c9ee7b5-0227-43c2-be55-909751dbd61a.htseq.counts.gz,lenght
ENSG00000167578,475,494,1008,265,325,278,582,613,357,361,...,269,458,1395,621,384,403,697,369,392,18722
ENSG00000078237,409,475,1082,660,804,535,1215,1226,751,675,...,651,154,812,674,73,395,598,1240,1090,46830
ENSG00000146083,3211,4264,5319,3457,3864,1257,3702,3043,4536,3259,...,2141,1071,2456,4049,1466,1933,2840,2657,5324,11328
ENSG00000158486,53,18,168,12,116,62,97,18,90,836,...,198,0,27,25,790,28,4,153,318,226330
ENSG00000198242,11411,20733,15310,32889,11328,27760,15145,25741,18640,27510,...,14479,12937,24780,27905,15907,4641,24878,11621,39908,4966


In [32]:
df = df.div(df['lenght'],axis=0)
df = df.mul(1000,axis=0) #kilobase
df.drop('lenght', axis=1, inplace=True)

In [35]:
df.head()

,ab30d37c-c510-496f-a704-6687ea39b847.htseq.counts.gz,e89c96d6-a2ab-4bac-bbba-8c52d49a3edf.htseq.counts.gz,d2587070-cb7d-440d-ae49-52f5077248e6.htseq.counts.gz,e733093b-8a60-46d7-9ddd-ec8a82206126.htseq.counts.gz,f02be12f-2e34-4089-be5c-0f81ca6038c8.htseq.counts.gz,6b424e38-1004-4434-be48-ee01dad67982.htseq.counts.gz,6f8c77f2-67de-411d-a03d-5fef3216b71b.htseq.counts.gz,3490b791-16ec-41e8-9015-5f3d33c856c0.htseq.counts.gz,97675f21-ce94-4e7c-bc46-179cc2903bf3.htseq.counts.gz,df692bf8-d523-4528-aa0c-a0549373adee.htseq.counts.gz,...,87968a34-cdba-4335-92d9-0f5b34ed27c9.htseq.counts.gz,32f7779d-9065-4241-ad44-efe90148ac23.htseq.counts.gz,358f2893-02b2-4d36-9c76-b818c28b6027.htseq.counts.gz,11797830-6218-43c6-8c41-515583575668.htseq.counts.gz,8677d42c-aef1-444d-b9a2-0dbcced0e886.htseq.counts.gz,fbef38b2-454d-44e6-b27f-5a790c6f8c17.htseq.counts.gz,c327832d-9cf8-4734-a745-42879ebe6702.htseq_counts.txt.gz,90d19c5c-f80b-454b-b776-99d13f01617e.htseq.counts,6d9c73d6-215a-49f9-bc9f-af019f8db55c.htseq.counts.gz,9c9ee7b5-0227-43c2-be55-909751dbd61a.htseq.counts.gz
ENSG00000167578,25.371221,26.386070,53.840402,14.154471,17.359256,14.848841,31.086422,32.742228,19.068476,19.282128,...,48.125200,14.368123,24.463198,74.511270,33.169533,20.510629,21.525478,37.228929,19.709433,20.937934
ENSG00000078237,8.733718,10.143071,23.104847,14.093530,17.168482,11.424301,25.944907,26.179799,16.036729,14.413837,...,10.356609,13.901345,3.288490,17.339312,14.392483,1.558830,8.434764,12.769592,26.478753,23.275678
ENSG00000146083,283.456921,376.412429,469.544492,305.173023,341.101695,110.963983,326.800847,268.626412,400.423729,287.694209,...,251.500706,189.000706,94.544492,216.807910,357.432910,129.413842,170.639124,250.706215,234.551554,469.985876
ENSG00000158486,0.234171,0.079530,0.742279,0.053020,0.512526,0.273936,0.428578,0.079530,0.397649,3.693722,...,1.877789,0.874829,0.000000,0.119295,0.110458,3.490479,0.123713,0.017673,0.676004,1.405028
ENSG00000198242,2297.825211,4174.989932,3082.964156,6622.835280,2281.111559,5590.012082,3049.738220,5183.447443,3753.523963,5539.669754,...,1689.085783,2915.626259,2605.114781,4989.931534,5619.210632,3203.181635,934.554974,5009.665727,2340.112767,8036.246476


In [34]:
df.to_csv("mainTable_RPK.csv",index=True, header=True)

### null

In [ ]:
df_null=pd.read_csv("nullTable.csv",header=None, index_col=None)
df_null.insert(0,'lenght',q_many['lenght'].values)
df_null.head()

In [ ]:
df_null = df_null.div(df_null['lenght'],axis=0).mul(1000,axis=0) #kilobase
df_null.drop('lenght',axis=1, inplace=True)

In [ ]:
df_null.head()

In [ ]:
df_null.to_csv("nullTable.csv", header=False, index=False)

## Means sigmas
thresholds are applied for FPKM between 0.1 and 1e5

In [ ]:
%timeit
means = []
for i,g in enumerate(df.loc[:,df.keys()[0]].values):
    means.append(np.average(df.loc[i,df.keys()[1:]].values))
    if(i%100==0):
        print("elapsed %d"%i)

In [ ]:
%timeit
variances = []
for i,g in enumerate(df.loc[:,df.keys()[0]].values):
    variances.append(np.var(df.loc[i,df.keys()[1:]].values))
    if(i%100==0):
        print("elapsed %d"%i)

In [ ]:
df_mv = pd.DataFrame(data=np.array(means), columns=['mean'], index=df.loc[:,df.keys()[0]].values)
df_mv.insert(1, 'variance', np.array(variances))
df_mv.head()
df_mv.to_csv("meanVariances.csv")

## manipulate meansigmas.csv

In [ ]:
df_mv = pd.read_csv("meanVariances.csv", index_col = [0])
df_mv.head()

### Add type of gene

In [ ]:
gdf = pd.read_csv("genes.txt", index_col=[0])
gdf.head()

In [ ]:
len(gdf)

In [ ]:
df_mv = df_mv.drop(labels='type_of_gene', axis=1)
df_mv.insert(1,'type_of_gene', '')
#for g in df_mv[[df_mv['type_of_gene']==type_of_gene]].index.values
for g in df_mv.index.values:
    df_mv.at[g, 'type_of_gene'] = gdf.loc[g[:15]]['type_of_gene']

In [ ]:
#df_mv = df_mv[df_mv['type_of_gene']==type_of_gene]
df_mv.to_csv("meanVariances.csv", index=True, header=True)

### Add occurence

In [ ]:
odf = pd.read_csv("O.dat", header=None)
#print(len(df_mv[df_mv['type_of_gene']=='protein-coding']))
odf.head()

In [ ]:
len(np.array(odf.loc[:,0].values*5000))

In [ ]:
type_of_gene = 'protein-coding'

In [ ]:
len(gdf[gdf['type_of_gene']==type_of_gene].index)

In [ ]:
len(df_mv.index)

In [ ]:
gdfpc = gdf.loc[gdf['type_of_gene']==type_of_gene]
gdfpc.insert(1, 'occurrence', np.array(odf.loc[:,0].values*5000))
#df_mv.insert(3,'occurrence', 0)
for g in df_mv.index.values:
    df_mv.at[g, 'occurrence'] = gdfpc.loc[g[:15]]['occurrence']

In [ ]:
df_mv.head()

In [ ]:
df_mv.to_csv("meanVariances.csv", header=True, index=True)

## clean floating points

In [ ]:
df.to_csv("mainTable.csv", index=False, header=True, float_format="%.1f")

# File list and properties

In [ ]:
#df_files = pd.read_csv("files.txt")
#df_files = pd.DataFrame(index=[sample[:36] for sample in df_files.columns.values[1:]])

In [ ]:
#df_files = pd.read_csv("files.txt", index_col=[0], header=None)
df_files.insert(0,'primary_site','')
df_files.insert(1,'disease_type', '')
df_files.head()

In [ ]:
#for sample in df_files.index:
for sample in df_files[df_files['primary_site']==''].index:
    try:
        sdf=queryFile(sample+'.FPKM.txt.gz')
        df_files.at[sample,'primary_site'] = sdf['primary_site'][0]
        df_files.at[sample, 'disease_type'] = sdf['disease_type'][0]
    except:
        for i in range(10):
            try:
                sdf=queryFile(sample+'%1.0f.FPKM.txt'%i)
                df_files.drop(labels=sample, axis=0, inplace=True)
                sample = sample+'%d'%i
                df_files.at[sample,'primary_site'] = sdf['primary_site'][0]
                df_files.at[sample, 'disease_type'] = sdf['disease_type'][0]
                break
            except:
                continue

In [ ]:
df_files.to_csv("files.txt", index=True, header=True)